In [2]:
import sys
sys.path.append("..")

from utils import *

# Chikwadraattoetsen

## Toepassing 1: onafhankelijkheidstoets van twee categorische variabelen

In [3]:
# Genereer LaTeX-tabellen
def dataframe_naar_latex(arr, row_labels, col_labels, caption):
    nrows, ncols = arr.shape
    alignments = f"c|{"c" * ncols}|c"
    header = f"""
        \\begin{{table}}[H]
            \\centering
            \\caption{{{caption}}}
            \\begin{{tabular}}{{{alignments}}}
                \\toprule
                    & {" & ".join(col_labels)} & Totaal \\\\
                \\midrule
    """

    latex_data = ""
    for i, row_label in enumerate(row_labels):
        latex_data += f"\t\t\t{row_label} {"".join( [f"& ${pretty_print(val)}$ " for val in arr[i,:]] )}& ${pretty_print(np.sum(arr[i,:]))}$ \\\\ \n"
                
    footer = f"""
                \\midrule
                    Totaal {"".join( [f"& ${pretty_print(val)}$ " for val in np.sum(arr, axis=0)] )}& ${pretty_print(np.sum(arr))}$ \\\\
                \\bottomrule
            \\end{{tabular}}
        \\end{{table}}
    """
    return header + latex_data + footer

def generate_latex_tables_chi2_onafhankelijkheid(observed, row_names=None, col_names=None):
    # Transform the numpy array to a pandas DataFrame
    chi2, p, dof, expected = chi2_contingency(observed, correction=False)
    # print(f"Toetsingsgrootheid: $\\chi^2 \\approx {pretty_print(chi2)}")
    # print(f"$p$-waarde: $p = P(X^2 \\ge \\chi^2) = \\chi^2\\text{{cdf}}(\\text{{lower}}={pretty_print(chi2)}; \\text{{upper}}=10^{{99}}; \\text{{df}}={dof}) \\approx {pretty_print(p)}$.")
    # print(f"Expected table: {expected}")

    nrows, ncols = expected.shape
    
    latex_code = f"""
    \\begin{{minipage}}{{0.45\\textwidth}}
        {dataframe_naar_latex(observed, row_names, col_names, "Geobserveerde frequenties (observed)")}
    \\end{{minipage}}
    \\hfill
    \\begin{{minipage}}{{0.45\\textwidth}}
        {dataframe_naar_latex(expected, row_names, col_names, "Verwachte frequenties (expected)")}
    \\end{{minipage}}
    """
    return latex_code, expected

def generate_latex_calculation_chi2_onafhankelijkheid(nrows, ncols, observed, expected):
    """
    Genereert een LaTeX-berekening met Observed, Expected.

    Parameters:
    - categories: lijst van categorieen.
    - observed: lijst van geobserveerde waarden.
    - expected: lijst van verwachte waarden.

    Returns:
    - Een string met de LaTeX-berekeningscode.
    """

    rows = list(range(1, nrows+1))
    cols = list(range(1, ncols+1))
    general_formula = " + ".join([f"\\frac{{(O_{{{row}, {col}}} - E_{{{row},{col}}})^2}}{{E_{{{row}, {col}}}}}" for (row, col) in it.product(rows, cols)])
    filled_in_formula = f" + ".join([f"\\frac{{({pretty_print(observed[i-1,j-1])} - {pretty_print(expected[i-1,j-1])})^2}}{{{pretty_print(expected[i-1,j-1])}}}" for (i,j) in it.product(rows, cols)])
    result = np.sum((observed - expected) ** 2 / expected) 
    
    latex_code = f"""
        \\begin{{align*}}
            \\chi^2 &= {general_formula} \\\\
                    &= {filled_in_formula} \\\\
                    &\\approx {pretty_print(result)}.
        \\end{{align*}}
    """
    return latex_code, result

def chi2_onafhankelijkheid(row_names, col_names, observed, alpha, num_estimated_parameters=0):
    """
    Voert een chikwadraat onafhankelijkheidstoets uit.
    """
    observed = np.asarray(observed)

    nrows = len(row_names)
    ncols = len(col_names)

    print(f"We voeren een chikwadraat toets voor onafhankelijkheid met een significantieniveau van $\\alpha = {alpha}$.")
    print(f"Allereerst berekenen we op basis van de aanname van onafhankelijkheid de verwachte frequenties:")
    latex_code, expected = generate_latex_tables_chi2_onafhankelijkheid(observed, row_names, col_names)
    print(latex_code)

    print(f"Vervolgens berekenen we de toetsingsgrootheid $X^2$ als volgt:")
    latex_code, chi2_stat = generate_latex_calculation_chi2_onafhankelijkheid(nrows, ncols, observed, expected)
    print(latex_code)

    p_value = 1 - chi2.cdf(chi2_stat, df=(nrows-1)*(ncols-1))

    print(f"De toetsingsgrootheid $X^2$ volgt onder de nulhypothese een $\\chi^2$-verdeling met $\\text{{df}}= (\\#\\text{{rijen}}-1)\\cdot(\\#\\text{{kolommen}}-1) = ({nrows}-1)\\cdot({ncols}-1) = {(nrows-1)*(ncols-1)}$ vrijheidsgraden.")
    print(f"""
        \\begin{{align*}}
            p   &= P(\\chi^2 > {pretty_print(chi2_stat)}) \\\\ 
                &= \\chi^2\\text{{cdf}}(\\text{{lower}}={pretty_print(chi2_stat)}; \\text{{upper}}=10^{{99}}; \\text{{df}}={(nrows-1)*(ncols-1)}) \\\\
                &\\approx {pretty_print(p_value)}.   
        \\end{{align*}}"""
    )
    if p_value < alpha:
        print(f"Omdat $p < \\alpha$, wordt de nulhypothese $H_0$ verworpen.")
        print(f"Er is voldoende reden om aan te nemen dat de twee variabelen afhankelijk zijn van elkaar.")
    else:
        print(f"Omdat $p > \\alpha$, wordt de nulhypothese $H_0$ aangenomen.")
        print(f"Er is onvoldoende reden om de aanname te verwerpen dat de twee variabelen onafhankelijk zijn van elkaar.")
        

In [4]:
observed = np.array([
    [52, 22],
    [23, 9],
    [38, 28]
])
nrows, ncols = observed.shape
row_names = ["Leiderschap", "Avontuur", "Vaderland dienen"]
col_names = ["Cadetten", "Adelborsten"]
alpha = 0.05

chi2_onafhankelijkheid(row_names, col_names, observed, alpha)

We voeren een chikwadraat toets voor onafhankelijkheid met een significantieniveau van $\alpha = 0.05$.
Allereerst berekenen we op basis van de aanname van onafhankelijkheid de verwachte frequenties:

    \begin{minipage}{0.45\textwidth}
        
        \begin{table}[H]
            \centering
            \caption{Geobserveerde frequenties (observed)}
            \begin{tabular}{c|cc|c}
                \toprule
                    & Cadetten & Adelborsten & Totaal \\
                \midrule
    			Leiderschap & $52$ & $22$ & $74$ \\ 
			Avontuur & $23$ & $9$ & $32$ \\ 
			Vaderland dienen & $38$ & $28$ & $66$ \\ 

                \midrule
                    Totaal & $113$ & $59$ & $172$ \\
                \bottomrule
            \end{tabular}
        \end{table}
    
    \end{minipage}
    \hfill
    \begin{minipage}{0.45\textwidth}
        
        \begin{table}[H]
            \centering
            \caption{Verwachte frequenties (expected)}
            \begin{tabular}{c|cc|c}
     

## Toepassing 2: aanpassingstoets (goodness-of-fit test)

In [5]:
def generate_latex_table_chi2_aanpassing(categories, observed, expected):
    """
    Genereert een LaTeX-tabel met drie kolommen: Categorie, Observed, Expected.

    Parameters:
    - categories: lijst van labels of namen van wat gemeten wordt.
    - observed: lijst van geobserveerde waarden.
    - expected: lijst van verwachte waarden.

    Returns:
    - Een string met de LaTeX-tabelcode.
    """    
    latex_code = f"""
        \\begin{{tabular}}{{lrr}}
            \\toprule
                \\textbf{{Categorie}} & \\textbf{{Observed}} & \\textbf{{Expected}} \\
            \\midrule
    """
    for cat, obs, exp in zip(categories, observed, expected):
        latex_code += f"\t\t{cat} & ${obs}$ & ${pretty_print(exp)}$ \\\\\n"

    latex_code += f"""
        \\bottomrule
    \\end{{tabular}}
    """
    return latex_code

def generate_latex_calculation_chi2_aanpassing(categories, observed, expected):
    """
    Genereert een LaTeX-berekening met Observed, Expected.

    Parameters:
    - categories: lijst van categorieen.
    - observed: lijst van geobserveerde waarden.
    - expected: lijst van verwachte waarden.

    Returns:
    - Een string met de LaTeX-berekeningscode.
    """
    latex_code = f"""
        \\begin{{align*}}
    """    

    general_formula = " + ".join([f"\\frac{{(O_{{{cat}}} - E_{{{cat}}})^2}}{{E_{{{cat}}}}}" for cat in categories])
    filled_in_formula = " + ".join([f"\\frac{{({pretty_print(observed[i])} - {pretty_print(expected[i])})^2}}{{{pretty_print(expected[i])}}}" for i in range(len(observed))])
    result = np.sum((observed - expected) ** 2 / expected) 
    latex_code += f"""
        \\chi^2 &= {general_formula}\\\\
                &= {filled_in_formula}\\\\
                &\\approx {pretty_print(result)}
    \\end{{align*}}
    """
    return latex_code, result

def chi2_aanpassing(categories, observed, expected, alpha, num_estimated_parameters=0):
    """
    Voert een chikwadraat aanpassingstoets uit.

    Parameters:
    - categories: list of array of categories
    - observed: list or array of observed frequencies
    - expected: list or array of expected frequencies (optional).
    
    Returns:
    - chi2_test_statistic: Chi-square statistic
    - p_value: corresponding p-value
    """
    categories = np.asarray(categories)
    observed = np.asarray(observed)
    expected = np.asarray(expected)

    print(f"We voeren een chikwadraat toets voor aanpassing met een significantieniveau van $\alpha = {alpha}$.")
    print(f"Allereerst berekenen we op basis van de gegeven kansverdeling de verwachte frequenties:")
    print(generate_latex_table_chi2_aanpassing(categories, observed, expected))

    print(f"Vervolgens berekenen we de toetsingsgrootheid $X^2$ als volgt:")
    latex_code, chi2_stat = generate_latex_calculation_chi2_aanpassing(categories, observed, expected)
    print(latex_code)

    p_value = 1 - chi2.cdf(chi2_stat, df=len(categories)-1-num_estimated_parameters)

    print(f"De toetsingsgrootheid $X^2$ volgt onder de nulhypothese een $\\chi^2$-verdeling met $\\text{{df}}= {len(categories)} - 1 - {num_estimated_parameters} = {len(categories)-1-num_estimated_parameters}$ vrijheidsgraden.")
    print(f"""
        \\begin{{align*}}
            p = P(\\chi^2 > {pretty_print(chi2_stat)}) &= \\chi^2\\text{{cdf}}(\\text{{lower}}={pretty_print(chi2_stat)}; \\text{{upper}}=10^{{99}}; \\text{{df}}={len(categories)-1-num_estimated_parameters}) \\\\
                                             &\\approx {pretty_print(p_value)}.   
        \\end{{align*}}"""
    )
    if p_value < alpha:
        print(f"Omdat $p < \\alpha$, wordt de nulhypothese $H_0$ verworpen.")
        print(f"Er is voldoende reden om aan te nemen dat de geobserveerde data geen realisaties van een andere kansverdeling dan de gegeven kansverdeling.")
    else:
        print(f"Omdat $p > \\alpha$, wordt de nulhypothese $H_0$ aangenomen.")
        print(f"Er is onvoldoende reden om de aanname te verwerpen dat de geobserveerde data tot stand zijn gekomen als trekkingen van de gegeven kansverdeling.")
           

    return chi2_stat, p_value



In [6]:
## DRIVER CODE ###

# Invoervariabelen
categories = np.array([0, 1, 2, 3, 4])
observed = np.array([15, 105, 290, 360, 230])
sum_observations = np.dot(categories, observed)
n_observations = np.sum(observed)
average_observation = sum_observations / n_observations
expected = np.array([n_observations * binom.pmf(k=k, n=4, p=0.7) for k in range(5)])
# expected = [100 * percentage / 100 for percentage in [28, 36, 24, 12]]
alpha=0.05
num_estimated_parameters = 0

# Generate the LaTeX code for the solution
chi2_aanpassing(categories, observed, expected, alpha, num_estimated_parameters)


We voeren een chikwadraat toets voor aanpassing met een significantieniveau van $lpha = 0.05$.
Allereerst berekenen we op basis van de gegeven kansverdeling de verwachte frequenties:

        \begin{tabular}{lrr}
            \toprule
                \textbf{Categorie} & \textbf{Observed} & \textbf{Expected} \
            \midrule
    		0 & $15$ & $8.1$ \\
		1 & $105$ & $75.6$ \\
		2 & $290$ & $264.6$ \\
		3 & $360$ & $411.6$ \\
		4 & $230$ & $240.1$ \\

        \bottomrule
    \end{tabular}
    
Vervolgens berekenen we de toetsingsgrootheid $X^2$ als volgt:

        \begin{align*}
    
        \chi^2 &= \frac{(O_{0} - E_{0})^2}{E_{0}} + \frac{(O_{1} - E_{1})^2}{E_{1}} + \frac{(O_{2} - E_{2})^2}{E_{2}} + \frac{(O_{3} - E_{3})^2}{E_{3}} + \frac{(O_{4} - E_{4})^2}{E_{4}}\\
                &= \frac{(15 - 8.1)^2}{8.1} + \frac{(105 - 75.6)^2}{75.6} + \frac{(290 - 264.6)^2}{264.6} + \frac{(360 - 411.6)^2}{411.6} + \frac{(230 - 240.1)^2}{240.1}\\
                &\approx 26.643
    \end{align

(26.643026825242533, 2.3470867858477185e-05)

### Plotting functions

In [7]:
def chi2_choose_domain(df):
    if df <= 5:
        xmin = 0.001
    else:
        xmin = 0

    xmax = chi2.ppf(0.999, df=df)
    return xmin, xmax

def chi2_calculate_yaxis_ub(y, df):
    # Set the top of y-axis to enhance visibility for lower degrees of freedom
    if df <= 2:
        top = 0.5
    else:
        top = max(y) * 1.1
    return top
    
def chi2_p_value(test_statistic, alpha, df, filename):
    xmin, xmax = chi2_choose_domain(df=df)
    x = np.linspace(xmin, xmax, 1000)
    y = chi2.pdf(x, df=df)
    maxy = chi2_calculate_yaxis_ub(y, df=df)
    
    # Kansberekening
    critical_value = chi2.ppf(q=1-alpha, df=df)
    p_value = 1 - chi2.cdf(test_statistic, df=df)

    # Plot
    fig, ax = plt.subplots(figsize=(8, 4))
    ax.plot(x, y)

    ymin, ymax = ax.get_ylim()
    ytext, offset = get_y_annotation(ax)
    ax.set_ylim(ymin - 2 * offset, ymax)
    
    if test_statistic:
        ax.fill_between(x, y, where=(x >= test_statistic), color=primary_plot_color, alpha=0.5, label=f"$p={p_value:.4f}$")
        ax.plot([test_statistic, test_statistic], [0, chi2.pdf(test_statistic, df=df)], color=primary_plot_color, linestyle='--')
        ax.text(test_statistic, ytext, "$\\chi^2$", color=primary_plot_color, ha="center")

    ax.fill_between(x, y, where=(x >= critical_value), color=critical_color, alpha=0.5, label=f"$\\alpha={alpha}$")
    ax.plot([critical_value, critical_value], [0, chi2.pdf(critical_value, df=df)], color=critical_color, linestyle='--')
    ax.text(critical_value, ytext, "$g$", color=critical_color, ha="center")
    
    ax.set_title(f'Chikwadraatverdeling met df$={df}$ {"vrijheidsgraad" if df == 1 else "vrijheidsgraden"}')
    ax.set_xlabel('$x$')
    ax.set_ylabel('Kansdichtheid $f(x)$')
    # ax.ylim(bottom=-0.12 * maxy, top=maxy)
    plt.tight_layout()
    plt.legend()
    plt.savefig(filename, format='png')


def chi2_critical_value(test_statistic, alpha, df, filename):
    xmin, xmax = chi2_choose_domain(df=df)
    x = np.linspace(xmin, xmax, 1000)
    y = chi2.pdf(x, df=df)
    maxy = chi2_calculate_yaxis_ub(y, df=df)
    
    # Kansberekening
    critical_value = chi2.ppf(q=1-alpha, df=df)
    
    # Plot
    fig, ax = plt.subplots(figsize=(8, 4))
    ax.plot(x, y)
    
    ymin, ymax = ax.get_ylim()
    ytext, offset = get_y_annotation(ax)
    ax.set_ylim(ymin - 2 * offset, ymax)

    if test_statistic:
        ax.plot([test_statistic, test_statistic], [0, chi2.pdf(test_statistic, df=df)], color=primary_plot_color, linestyle='--', label=f"$\\chi^2={test_statistic:.4f}$")
        ax.text(test_statistic, ytext, "$\\chi^2$", color=primary_plot_color, ha="center")

    ax.fill_between(x, y, where=(x >= critical_value), color=critical_color, alpha=0.3, label=f"Kritiek gebied: $[g,\\infty) = [{critical_value:.4f}: \\infty)$")
    ax.plot([critical_value, critical_value], [0, chi2.pdf(critical_value, df=df)], color=critical_color, linestyle='--')
    ax.text(critical_value, ytext, "$g$", color=critical_color, ha="center")
    
    ax.set_title(f'Chikwadraatverdeling met df$={df}$ {"vrijheidsgraad" if df == 1 else "vrijheidsgraden"}')
    ax.set_xlabel('$x$')
    ax.set_ylabel('Kansdichtheid $f(x)$')
    # ax.ylim(bottom=-0.12 * maxy, top=maxy)
    
    plt.legend()
    plt.tight_layout()
    plt.savefig(filename, format='png')


### Voorbeeld gebruik

In [8]:
# chi2_critical_value(test_statistic=None, alpha=0.05, df=6, filename=FIGURE_PATH + "chisq_test1.png")

# chi2_critical_value(test_statistic=12.0660, alpha=0.01, df=2, filename=FIGURE_PATH + "chisq_test2.png")

# chi2_p_value(test_statistic=4.6402, alpha=0.01, df=1, filename=FIGURE_PATH + "chisq_test3.png")

# chi2_critical_value(test_statistic=10.5717, alpha=0.05, df=3, filename=FIGURE_PATH + "chisq_test4.png")
# chi2_p_value(test_statistic=10.5717, alpha=0.05, df=3, filename=FIGURE_PATH + "chisq_test5.png")

# chi2_critical_value(test_statistic=10.1366, alpha=0.05, df=3, filename=FIGURE_PATH + "chisq_test6.png")
# chi2_p_value(test_statistic=10.1366, alpha=0.05, df=3, filename=FIGURE_PATH + "chisq_test7.png")